<a href="https://colab.research.google.com/github/Rain1618/Hide-It/blob/main/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm, model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [12]:
df = pd.read_csv('/content/cleaned_HideIt_data.csv')
df.head()

,subreddit,trigger,token_text
0,alcoholism,safe,"['could', 'someone', 'send', 'me', 'a', 'messa..."
1,alcoholism,safe,"['a', 'rock', 'bottom', 'story']"
2,alcoholism,addiction,"['does', 'this', 'video', 'infuriate', 'anyone..."
3,alcoholism,addiction,"['how', 'do', 'i', 'feel', 'joy', 'when', 'i',..."
4,alcoholism,addiction,"['what', 'to', 'do', 'i', 'just', 'hit', '5', ..."


In [25]:
df = pd.read_csv('/content/untokenized_HideIt_data.csv')
df.head()

,subreddit,trigger,text
0,alcoholism,safe,could someone send me a message? i'm having a ...
1,alcoholism,safe,a rock bottom story
2,alcoholism,addiction,does this video infuriate anyone else or is it...
3,alcoholism,addiction,how do i feel joy when i'm not drunk? i have a...
4,alcoholism,addiction,what to do i just hit 5 months sober which is ...


In [30]:
import spacy
nlp = spacy.load('en_core_web_sm')

df['new_col'] = df['text'].apply(lambda x: nlp(x))

In [38]:
df.rename(columns={'new_col': 'token_text'}, inplace=True)
df.drop(['text'], axis = 1, inplace=True)
df.head()

,subreddit,trigger,token_text
0,alcoholism,safe,"(could, someone, send, me, a, message, ?, i, '..."
1,alcoholism,safe,"(a, rock, bottom, story)"
2,alcoholism,addiction,"(does, this, video, infuriate, anyone, else, o..."
3,alcoholism,addiction,"(how, do, i, feel, joy, when, i, 'm, not, drun..."
4,alcoholism,addiction,"(what, to, do, i, just, hit, 5, months, sober,..."


In [40]:
df.to_csv('spacy_HideIt_data.csv')

In [32]:
df = df[df['trigger'].isin(['safe', 'addiction','abuse','sexual violence', 'suicide', 'eating disorder'])] 

In [33]:
X = df['new_col'].astype(str)
y = df['trigger'].astype(str)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [15]:
y.unique()

array(['safe', 'addiction', 'abuse', 'sexual violence', 'eating disorder',
       'suicide'], dtype=object)

In [34]:
classes_names = ['safe', 'addiction', 'abuse', 'sexual violence', 'eating disorder',
       'suicide']

encoder = LabelEncoder()
encoder.fit(classes_names)
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

In [35]:
#Tfidf 
Tfidf_vect = TfidfVectorizer(max_features=500) 
Tfidf_vect.fit(X)
X_train = Tfidf_vect.transform(X_train)
X_test = Tfidf_vect.transform(X_test)

###Testing difference models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# Define a list of models to test
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision tree': DecisionTreeClassifier(),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    'MultinminalNB': MultinomialNB(),
    'Linear SVC': svm.LinearSVC(random_state=42)
}

# Iterate over the models
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name}: Accuracy = {accuracy}')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Accuracy = 0.7719953996549741
Decision tree: Accuracy = 0.5948821161587119
Support Vector Machine: Accuracy = 0.7765238642898218
Random Forest: Accuracy = 0.7398648648648649
MultinminalNB: Accuracy = 0.723188614146061
Linear SVC: Accuracy = 0.7778177113283496


In [36]:
clf = svm.LinearSVC(random_state=42).fit(X_train, y_train)
clf.score(X_test, y_test)

0.7763082231167338

In [37]:
from sklearn.model_selection import cross_val_score
clf = svm.LinearSVC(random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=5)

In [21]:
scores

array([0.78054624, 0.76707405, 0.76851186, 0.77090822, 0.78049365])

In [24]:
import pickle
pickle.dump(clf, open("LinearSVC_model.sav", 'wb'))

In [41]:
import joblib

# Assuming your trained SVM model is stored in the variable 'svm_model'
joblib.dump(clf, 'LinearSVC_model2.joblib')

['LinearSVC_model2.joblib']